In [ ]:
# 📦 Imports
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier, VotingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from xgboost import XGBClassifier

# 🧾 1. Load the Dataset
df = pd.read_csv("/content/dataset.csv")  # Replace with your actual filename

# 🛠️ 2. Feature Engineering
df['pixel_density'] = df['px_height'] * df['px_width']
df['screen_area'] = df['sc_h'] * df['sc_w']
df['camera_quality'] = df['pc'] + df['fc']

# 🎯 3. Prepare Features and Target
X = df.drop(['price_range'], axis=1)
y = df['price_range']

# 📊 Optional: Visualize Correlation
# plt.figure(figsize=(14, 10))
# sns.heatmap(df.corr(), annot=True, cmap="coolwarm")
# plt.title("Feature Correlation")
# plt.show()

# 📤 4. Train-Test Split
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, stratify=y, random_state=42
)

# 🔃 5. Feature Scaling (for logistic regression only, not needed for trees/XGBoost)
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# 🚀 6. Train XGBoost Classifier
xgb_model = XGBClassifier(
    objective='multi:softmax',
    num_class=4,
    learning_rate=0.1,
    max_depth=7,
    n_estimators=300,
    subsample=0.8,
    colsample_bytree=0.8,
    use_label_encoder=False,
    eval_metric='mlogloss',
    random_state=42
)
xgb_model.fit(X_train, y_train)
xgb_preds = xgb_model.predict(X_test)

print("\n📈 XGBoost Model Accuracy:", accuracy_score(y_test, xgb_preds))
print("\n📊 Classification Report:\n", classification_report(y_test, xgb_preds))

# 🧠 7. Voting Classifier (Random Forest + Logistic Regression + XGBoost)
voting_model = VotingClassifier(estimators=[
    ('rf', RandomForestClassifier(n_estimators=200, random_state=42)),
    ('xgb', XGBClassifier(use_label_encoder=False, eval_metric='mlogloss', random_state=42)),
    ('lr', LogisticRegression(max_iter=2000))
], voting='hard')

voting_model.fit(X_train_scaled, y_train)
voting_preds = voting_model.predict(X_test_scaled)

print("\n📈 Voting Classifier Accuracy:", accuracy_score(y_test, voting_preds))
print("\n📊 Voting Classification Report:\n", classification_report(y_test, voting_preds))

# 🔁 8. Cross-validation (Optional)
cv_scores = cross_val_score(xgb_model, X, y, cv=5)
print("\n🔁 XGBoost Cross-validation Accuracy: {:.2f}%".format(cv_scores.mean() * 100))


/usr/local/lib/python3.11/dist-packages/xgboost/core.py:158: UserWarning: [18:32:32] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)



📈 XGBoost Model Accuracy: 0.9275

📊 Classification Report:
               precision    recall  f1-score   support

           0       0.96      0.99      0.98       100
           1       0.94      0.90      0.92       100
           2       0.87      0.90      0.88       100
           3       0.95      0.92      0.93       100

    accuracy                           0.93       400
   macro avg       0.93      0.93      0.93       400
weighted avg       0.93      0.93      0.93       400



/usr/local/lib/python3.11/dist-packages/xgboost/core.py:158: UserWarning: [18:32:34] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)



📈 Voting Classifier Accuracy: 0.9375

📊 Voting Classification Report:
               precision    recall  f1-score   support

           0       0.98      0.98      0.98       100
           1       0.93      0.93      0.93       100
           2       0.90      0.89      0.89       100
           3       0.94      0.95      0.95       100

    accuracy                           0.94       400
   macro avg       0.94      0.94      0.94       400
weighted avg       0.94      0.94      0.94       400



/usr/local/lib/python3.11/dist-packages/xgboost/core.py:158: UserWarning: [18:32:34] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)
/usr/local/lib/python3.11/dist-packages/xgboost/core.py:158: UserWarning: [18:32:35] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)
/usr/local/lib/python3.11/dist-packages/xgboost/core.py:158: UserWarning: [18:32:36] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)
/usr/local/lib/python3.11/dist-packages/xgboost/core.py:158: UserWarning: [18:32:37] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)
/usr/local/lib/python3.11/dist-packages/xgboost/core.py:158: UserWarning: [18:32:38] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_e


🔁 XGBoost Cross-validation Accuracy: 91.80%


In [ ]:
import joblib

# Save model and scaler
joblib.dump(voting_model, 'best_mobile_price_model.pkl')
joblib.dump(scaler, 'scaler.pkl')

print("✅ Model and scaler saved successfully.")


✅ Model and scaler saved successfully.


In [ ]:
from google.colab import files

files.download('best_mobile_price_model.pkl')
files.download('scaler.pkl')


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
# Load model and scaler
model = joblib.load('best_mobile_price_model.pkl')
scaler = joblib.load('scaler.pkl')

# Corrected input feature: use 'm_dep' instead of 'm_deep'
sample = pd.DataFrame([{
    'battery_power': 1000,
    'blue': 1,
    'clock_speed': 1.2,
    'dual_sim': 1,
    'fc': 2,
    'four_g': 1,
    'int_memory': 16,
    'm_dep': 0.5,  #
    'mobile_wt': 150,
    'n_cores': 4,
    'pc': 10,
    'px_height': 600,
    'px_width': 1000,
    'ram': 2048,
    'sc_h': 10,
    'sc_w': 5,
    'talk_time': 10,
    'three_g': 1,
    'touch_screen': 1,
    'wifi': 1,
    'pixel_density': 600000,
    'screen_area': 50,
    'camera_quality': 12
}])

# Apply same scaling
sample_scaled = scaler.transform(sample)

# Predict
predicted_class = model.predict(sample_scaled)
print("Predicted price range:", predicted_class[0])  # 0 to 3


Predicted price range: 1


In [ ]:
import os

os.makedirs("demos/mobile_price_predictor", exist_ok=True)


In [ ]:
%%writefile demos/mobile_price_predictor/app.py
import gradio as gr
import joblib
import pandas as pd
from model import predict_price_range

# Load class names
with open("class_names.txt", "r") as f:
    class_names = [line.strip() for line in f.readlines()]

# Gradio UI Input Fields
inputs = [
    gr.Number(label=col) for col in [
        'battery_power', 'blue', 'clock_speed', 'dual_sim', 'fc', 'four_g',
        'int_memory', 'm_dep', 'mobile_wt', 'n_cores', 'pc', 'px_height',
        'px_width', 'ram', 'sc_h', 'sc_w', 'talk_time', 'three_g',
        'touch_screen', 'wifi'
    ]
]

def gradio_predict(*args):
    sample = pd.DataFrame([args], columns=[
        'battery_power', 'blue', 'clock_speed', 'dual_sim', 'fc', 'four_g',
        'int_memory', 'm_dep', 'mobile_wt', 'n_cores', 'pc', 'px_height',
        'px_width', 'ram', 'sc_h', 'sc_w', 'talk_time', 'three_g',
        'touch_screen', 'wifi'
    ])
    pred_class, probs = predict_price_range(sample)
    return {class_names[i]: float(probs[i]) for i in range(len(class_names))}

examples = [["1000", "1", "1.2", "1", "2", "1", "16", "0.5", "150", "4", "10", "600", "1000", "2048", "10", "5", "10", "1", "1", "1"]]

demo = gr.Interface(
    fn=gradio_predict,
    inputs=inputs,
    outputs=gr.Label(num_top_classes=4, label="Predicted Price Range"),
    title="📱 Mobile Price Predictor",
    description="Enter the specifications of a mobile phone to predict its price range (0: Low, 3: High).",
    examples=examples
)

demo.launch()


Writing demos/mobile_price_predictor/app.py


In [ ]:
%%writefile demos/mobile_price_predictor/model.py
import joblib
import pandas as pd

# Load model and scaler
model = joblib.load("best_mobile_price_model.pkl")
scaler = joblib.load("scaler.pkl")

def predict_price_range(sample_df):
    sample_df["pixel_density"] = sample_df["px_height"] * sample_df["px_width"]
    sample_df["screen_area"] = sample_df["sc_h"] * sample_df["sc_w"]
    sample_df["camera_quality"] = sample_df["pc"] + sample_df["fc"]

    # Reorder columns as per model training
    sample_scaled = scaler.transform(sample_df)
    pred_class = model.predict(sample_scaled)
    pred_probs = model.predict_proba(sample_scaled)

    return pred_class[0], pred_probs[0]


Writing demos/mobile_price_predictor/model.py


In [ ]:
%%writefile demos/mobile_price_predictor/requirements.txt
pandas
numpy
scikit-learn
xgboost
gradio
joblib


Writing demos/mobile_price_predictor/requirements.txt


In [ ]:
%%writefile demos/mobile_price_predictor/class_names.txt
Low Cost (0)
Medium Cost (1)
High Cost (2)
Very High Cost (3)


Writing demos/mobile_price_predictor/class_names.txt


In [ ]:
from pathlib import Path
import json

# Create example directory
example_dir = Path("demos/mobile_price_predictor/examples")
example_dir.mkdir(parents=True, exist_ok=True)

# Example input
example_data = {
    "battery_power": 1000,
    "blue": 1,
    "clock_speed": 1.2,
    "dual_sim": 1,
    "fc": 2,
    "four_g": 1,
    "int_memory": 16,
    "m_dep": 0.5,
    "mobile_wt": 150,
    "n_cores": 4,
    "pc": 10,
    "px_height": 600,
    "px_width": 1000,
    "ram": 2048,
    "sc_h": 10,
    "sc_w": 5,
    "talk_time": 10,
    "three_g": 1,
    "touch_screen": 1,
    "wifi": 1
}

# Save example
with open(example_dir / "example_1.json", "w") as f:
    json.dump(example_data, f)


In [ ]:
# Zip the mobile_price_predictor folder but exclude cache and notebook files
!cd demos/mobile_price_predictor && zip -r ../mobile_price_predictor.zip * -x "*.pyc" "*.ipynb" "*__pycache__*" "*ipynb_checkpoints*"

try:
    from google.colab import files
    files.download("demos/mobile_price_predictor.zip")
except:
    print("Not running in Google Colab, can't use google.colab.files.download()")


  adding: app.py (deflated 55%)
  adding: class_names.txt (deflated 29%)
  adding: examples/ (stored 0%)
  adding: examples/example_1.json (deflated 41%)
  adding: model.py (deflated 56%)
  adding: requirements.txt (stored 0%)


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>